Перед запуском убедитесь, что в корне проекта есть файл .env и в нем заполнены выданные вам креды подключения к базам данных и хранилищу

In [2]:
%load_ext autoreload
%autoreload 2

In [1]:
import os
import pandas as pd
from dotenv import load_dotenv, find_dotenv
from sqlalchemy import create_engine, MetaData, Table


In [2]:
# подгружаем .env
load_dotenv()

True

In [3]:
# Считываем все креды
src_host = os.environ.get('DB_SOURCE_HOST')
src_port = os.environ.get('DB_SOURCE_PORT')
src_username = os.environ.get('DB_SOURCE_USER')
src_password = os.environ.get('DB_SOURCE_PASSWORD')
src_db = os.environ.get('DB_SOURCE_NAME') 

dst_host = os.environ.get('DB_DESTINATION_HOST')
dst_port = os.environ.get('DB_DESTINATION_PORT')
dst_username = os.environ.get('DB_DESTINATION_USER')
dst_password = os.environ.get('DB_DESTINATION_PASSWORD')
dst_db = os.environ.get('DB_DESTINATION_NAME')

s3_bucket = os.environ.get('S3_BUCKET_NAME')
s3_access_key = os.environ.get('AWS_ACCESS_KEY_ID')
s3_secret_access_key = os.environ.get('AWS_SECRET_ACCESS_KEY')

mle_tlg_token = os.environ.get('MLE_TELEGRAM_TOKEN')
mle_tlg_chat_id = os.environ.get('MLE_TELEGRAM_CHAT_ID')

In [4]:
# Создадим соединения
# src_conn = create_engine(f'postgresql://{src_username}:{src_password}@{src_host}:{src_port}/{src_db}')
dst_conn = create_engine(f'postgresql://{dst_username}:{dst_password}@{dst_host}:{dst_port}/{dst_db}')

In [5]:
# Укажем таблицу для удаления
TABLE_NAME = 'merged_flats_dataset'

In [ ]:
# Закроем соединения в конце работы
#src_conn.dispose()
dst_conn.dispose()

In [7]:
from sqlalchemy import inspect

# Удаление таблицы
def drop_table(table_name: str, engine):
    try:
        # Создаем объект метаданных
        metadata = MetaData()

        # Получаем информацию о таблице
        inspector = inspect(engine)
        if inspector.has_table(table_name):
            # Если таблица существует - удаляем
            table = Table(table_name, metadata, autoload_with=engine)
            table.drop(engine)
            print(f"Таблица {table_name} успешно удалена")
        else:
            print(f"Таблица {table_name} не существует")
            
    except Exception as e:
        print(f"Ошибка при удалении таблицы: {str(e)}")
        raise

# Проверка существования таблицы
def check_table_existence(table_name: str, engine):
    inspector = inspect(engine)
    exists = inspector.has_table(table_name)
    print(f"Таблица {table_name} существует: {exists}")
    return exists

# Выполнение операций
try:
    # Удаление таблицы
    drop_table(TABLE_NAME, dst_conn)
    
    # Проверка удаления
    check_table_existence(TABLE_NAME, dst_conn)

except Exception as e:
    print(f"Произошла ошибка: {str(e)}")
finally:
    # Закрытие соединения
    dst_conn.dispose()
    print("Соединение с базой закрыто")

Таблица merged_flats_dataset не существует
Таблица merged_flats_dataset существует: False
Соединение с базой закрыто
